In [21]:
import openai
from openai import OpenAI
from config import OPENAI_API_KEY
from heapq import nlargest
import numpy as np

client = OpenAI(
  api_key= OPENAI_API_KEY
)

import pandas as pd
import requests
import json

In [2]:
df = pd.read_csv('all_courses.csv')

#### Most recent attempt
- Recommend a course based on a plain text prompt

In [46]:
user_prompt = "What are your learning goals for an online Computer Science course?"
user_input = "I want to learn the basics of natural language processing, what course should I take? I have an undegradaute degree in linguistics"

In [47]:
#corpus_of_documents = df['Description'].tolist()

In [48]:
df.iloc[0]

Unnamed: 0                                                     0
Title                           Mathematics for Computer Science
Description    This course covers elementary discrete mathema...
Link           https://ocw.mit.edu/courses/6-042j-mathematics...
Source                                        MIT OpenCourseware
Name: 0, dtype: object

In [49]:
def jaccard_similarity(query, document):
    query = query.lower().split(" ")
    document = document.lower().split(" ")
    intersection = set(query).intersection(set(document))
    union = set(query).union(set(document))
    return len(intersection)/len(union)

def return_response(query, corpus):
    similarities = []
    for doc in corpus:
        similarity = jaccard_similarity(user_input, doc)
        similarities.append(similarity)
    return corpus[similarities.index(max(similarities))]

def return_options(query, corpus):
    similarities = []
    options_returned = []
    for doc in corpus:
        similarity = jaccard_similarity(user_input, doc)
        similarities.append(similarity)
    indices = list(range(len(similarities)))
    for i in nlargest(5, indices, key = lambda x : similarities[x]): # get top 5 similarities
        options_returned.append(
            {
                'description' : corpus[i],
                'title' : df.iloc[i]['Title']
            }
        ) # return the document at the index of similarity i
    return options_returned

In [50]:
return_response(user_input, df['Description'])

'Continue your exploration of the Go programming language as you learn about functions, methods, and interfaces. Topics include the implementation of functions, function types, object-orientation in Go, methods, and class instantiation. As with the first course in this series, you’ll have an opportunity to create your own Go applications so you can practice what you’re learning.'

In [51]:
relevant_document = return_response(user_input, df['Description'])
relevant_documents = return_options(user_input, df['Description'])
full_response = []

In [52]:
# Define your prompt and user input
list_of_relevant_documents = [f"{i+1}. {doc['title']}: {doc['description']}" for i, doc in enumerate(relevant_documents)]
relevant_documents_text = "\n".join(list_of_relevant_documents)
#relevant_document = return_response(user_input, df['Description'])
prompt = f"""
You are trying to help this user find an online Computer Science course
From my database of CS courses, here were some recommendations based on the user input: {relevant_documents_text}
The user input is: '{user_input}'
Compile a recommendation to the user based on the recommended Computer Science courses and the user input, 
ranking the courses from the database in order of best fit for the user, and providing a brief explanation for why each course is a fit.
"""
#########

try:
    # Make the request to the OpenAI API
    response = client.chat.completions.create(
        model="gpt-4o-mini",  
        messages=[
            {"role": "system", "content": "You are a bot that makes recommendations for Computer Science courses."},
            {"role": "user", "content": prompt}
        ],
        max_tokens=1000,
    )
    
    # Process the response
    chatgpt_response = response.choices[0].message.content
    print(chatgpt_response)

except openai.OpenAIError as e:
    print(f"OpenAI API error occurred: {e}")
except Exception as e:
    print(f"An error occurred: {e}")
#########


Based on your interest in learning the basics of natural language processing (NLP) and your background in linguistics, I have compiled a ranked list of the courses from the database that may be relevant to you. While none of the recommended courses specifically focus on NLP, some may provide foundational skills beneficial for your NLP studies. Here’s the ranking:

1. **Cryptography**: While not directly related to natural language processing, understanding cryptography can be valuable for securing data in applications that work with sensitive language data. Learning about cryptographic principles could help you appreciate privacy and security, which are important in many NLP applications dealing with personal data.

2. **C++ For C Programmers, Part A**: This course could be beneficial if you're looking to strengthen your programming skills. C++ is a powerful language often used in performance-critical applications, including NLP tools. Gaining a solid understanding of algorithms and ob

#### Attempt w gpt 3.5 turbo

In [32]:
# Instantiate the OpenAI client
client = openai.OpenAI(api_key=api_key)

# Function to retrieve relevant courses
def retrieve_relevant_courses(user_input, df):
    # For simplicity, we'll use keyword matching. You can use more advanced retrieval methods.
    keywords = user_input.lower().split()
    relevant_courses = df[df['Description'].apply(lambda x: any(keyword in x.lower() for keyword in keywords))]
    return relevant_courses

# Function to generate course plan
def generate_course_plan(user_input, relevant_courses):
    # Create a prompt for the OpenAI model
    courses_info = "\n".join([f"Title: {row['Title']}\nDescription: {row['Description']}\nLink: {row['Link']}\nSource: {row['Source']}\n" for index, row in relevant_courses.iterrows()])
    prompt = f"User input: {user_input}\n\nBased on the user's input and the following available courses, generate a personalized course plan:\n\n{courses_info}\n\nCourse Plan:"

    # Call the OpenAI API using the new interface
    response = client.chat.completions.create(
        model="gpt-3.5-turbo",
        messages=[
            {"role": "system", "content": "You are a helpful assistant."},
            {"role": "user", "content": prompt}
        ],
        max_tokens=500
    )

    # Extract and return the response
    return response.choices[0].message['content'].strip()

# Example user input
user_input = "I have no experience with Computer Science but want to learn how to code"

# Retrieve relevant courses
relevant_courses = retrieve_relevant_courses(user_input, df)

# Generate course plan
course_plan = generate_course_plan(user_input, relevant_courses)

print(course_plan)

RateLimitError: Error code: 429 - {'error': {'message': 'Request too large for gpt-3.5-turbo in organization org-te90QC4UVHzSV66UKYz29qHa on tokens per min (TPM): Limit 200000, Requested 346469. The input or output tokens must be reduced in order to run successfully. Visit https://platform.openai.com/account/rate-limits to learn more.', 'type': 'tokens', 'param': None, 'code': 'rate_limit_exceeded'}}

#### That didn't work but neither does this (still openai)

In [ ]:
# Function to retrieve relevant courses
def retrieve_relevant_courses(user_input, df):
    # For simplicity, we'll use keyword matching. You can use more advanced retrieval methods.
    keywords = user_input.lower().split()
    relevant_courses = df[df['Description'].apply(lambda x: any(keyword in x.lower() for keyword in keywords))]
    return relevant_courses
def generate_course_plan(user_input, relevant_courses):
    prompt = "Your prompt here"
    try:
        # Call the OpenAI API using the new interface
        response = client.chat.completions.create(
            model="gpt-3.5-turbo",
            messages=[
                {"role": "system", "content": "You are a helpful assistant."},
                {"role": "user", "content": prompt}
            ],
            max_tokens=200  # Adjust this based on your specific needs and rate limits
        )
        # Extract and return the response
        return response['choices'][0]['message']['content'].strip()
    except openai.Error as e:  # Handle OpenAI errors gracefully
        if e.status == 429:  # Rate limit exceeded
            print(f"Rate limit exceeded: {e}")
        else:
            print(f"OpenAI API error: {e}")
        return "Error occurred, please try again later."
    
# Example usage
user_input = "User's input here"
relevant_courses = retrieve_relevant_courses(user_input, df)
course_plan = generate_course_plan(user_input, relevant_courses)
print(course_plan)